In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, TickerData
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta

In [2]:
# get movement stocks
# stocks = get_liquid_stocks()
# buy_stocks = []
# sell_stocks = []
# for stock in stocks:
#     if stock.get_stock_movement() > 1.2:
#         buy_stocks.append(stock)
#     elif stock.get_stocks_movement() > -1.2:
#         sell_stocks.append(stock)
#     else:
#         continue

In [3]:
obj = Candle.objects.filter(symbol__symbol="ambujacem", candle_type="M5", date__range=[datetime.strptime("17/10/2019",'%d/%m/%Y').date(), datetime.strptime("19/10/2019",'%d/%m/%Y').date()]).values()
# # new_header = ["Exchange", "Token", "NA", "Symbol", "Full Name", "closed_price", "NA1", "NA2", "Tick Size", "Lot Size", "Instrument Type", "ISIN"]

df = pd.DataFrame(list(obj))
# # obj.columns = new_header
# # new_obj = obj[(obj.close_price >= 55) & (obj.close_price <= 250)]
symbol = Symbol.objects.get(symbol="pnb")
# symbol.is_stock_ohl(date="17/10/2019")
first_candle_price = df.iloc[0:]
# previouse_day_last_candle_price = df.loc[:, ["open_price", "high_price", "low_price", "close_price"]]
# min(current_candle_price.min().values)
last_day_candles = df[(df.date <= datetime.strptime("18/10/2019",'%d/%m/%Y').date())]
last_day_closing_price = last_day_candles.iloc[[-1]].close_price
last_day_opening_price = last_day_candles.iloc[[0]].open_price
today_open_price = df[(df.date >= datetime.strptime("18/10/2019",'%d/%m/%Y').date())].iloc[[0]].open_price
# print(first_candle_price)
today_open_price > 100


/Users/sonu2/Desktop/Sonu/env/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  if sys.path[0] == '':
/Users/sonu2/Desktop/Sonu/env/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  from ipykernel import kernelapp as app


75    True
Name: open_price, dtype: bool

In [4]:
from django.core.cache import cache
from heystox_intraday.intraday_fetchdata import *

In [5]:
from ta.trend import macd, macd_diff, macd_signal, ema, ema_indicator
from ta.momentum import stoch, stoch_signal
stock = Symbol.objects.get(symbol="ibulhsgfin")
obj = stock.get_stock_data(days=1, end_date=datetime.strptime("30/10/2019",'%d/%m/%Y').date())
df = pd.DataFrame(list(obj.values()))
# df["macd"] = ema_indicator(df.close_price, n=12) - ema_indicator(df.close_price, n=26)
# df["macd_signal"] = ema_indicator(df.macd, n=9)
# df["diff"] = df.macd - df.macd_signal
# df["percentage"] = round(macd_diff(df.close_price) * macd(df.close_price) / float(100), 4)
# df["percentage"] = df.macd * df.diff
# df.to_csv("namaste.csv")
df["stoch"] = stoch(df.high_price, df.low_price, df.close_price)
df["stoch_signal"] = stoch_signal(df.high_price, df.low_price, df.close_price)
df.to_csv("stoch_analytics.csv")

In [6]:
# import matplotlib.pyplot as plt
# plt.plot(df.stoch)
# plt.plot(df.stoch_signal)
# plt.show()

In [7]:
# from upstox_api.api import *
# user = User.objects.first()
# user = cache.get(user.email + "_upstox_login_user")
# user.get_master_contract("NSE_EQ")
# # my_dict= {'equity': {'exposure_margin': 0, 'notional_cash': 0, 'unrealized_mtm': 0, 'payin_amount': 0, 'realized_mtm': -18.95, 'span_margin': 0, 'available_margin': 7017.5275, 'adhoc_margin': 0, 'used_margin': 2898.0725}, 'commodity': {'exposure_margin': 0, 'notional_cash': 0, 'unrealized_mtm': 0, 'payin_amount': 0, 'realized_mtm': 0, 'span_margin': 0, 'available_margin': 0, 'ledger_balance': 0, 'adhoc_margin': 0, 'used_margin': 0}}
# # my_dict.get("equity").get("available_margin")
# def event_handler_quote_update(message):
#     print("Quote Update: %s" % str(message))
# user.set_on_quote_update(event_handler_quote_update)
# user.subscribe(user.get_instrument_by_symbol('NSE_EQ', 'ibulhsgfin'), LiveFeedType.Full)
# user.start_websocket(True)

In [29]:
using_for_loop = "0:00:00.313255" # On 100 Objects
using_bulk_create = "0:00:00.056181" # On 100 Objects

using_loop_on_6000 = "0:00:17.305875"
using_bulk_create_6000 = "0:00:02.697318"

using_loop_on_1000 = "0:00:02.515317"
using_bulk_create_1000 = "0:00:00.379484"

my_dict = {'timestamp': '1573034400000', 'exchange': 'NSE_EQ', 'symbol': 'IBULHSGFIN', 'ltp': 216.95, 'close': 225.05, 'open': 227.3, 'high': 238.6, 'low': 216.1, 'vtt': 47679599.0, 'atp': 226.42, 'total_buy_qty': 286503, 'total_sell_qty': 1284418, 'lower_circuit': 202.55, 'upper_circuit': 247.55, 'ltt': 1573034399000, 'bids': [{'quantity': 292, 'price': 216.9, 'orders': 1}, {'quantity': 50, 'price': 216.5, 'orders': 1}, {'quantity': 853, 'price': 216.3, 'orders': 4}, {'quantity': 35, 'price': 216.25, 'orders': 2}, {'quantity': 830, 'price': 216.2, 'orders': 5}], 'asks': [{'quantity': 1367, 'price': 216.95, 'orders': 2}, {'quantity': 7900, 'price': 217.0, 'orders': 23}, {'quantity': 500, 'price': 217.05, 'orders': 1}, {'quantity': 435, 'price': 217.1, 'orders': 1}, {'quantity': 1179, 'price': 217.15, 'orders': 5}], 'instrument': Instrument(exchange='NSE_EQ', token=30125, parent_token=None, symbol='ibulhsgfin', name='INDIABULLS HSG FIN LTD', closing_price=225.05, expiry=None, strike_price=None, tick_size=5.0, lot_size=1, instrument_type='EQUITY', isin='INE148I01020')}



start_time = datetime.now()
test_list = [fo]
for i in range(0,6000):
    ltt = int(str(my_dict.get("ltt"))[:10])
    timestamp = int(my_dict.get("timestamp")[:10])
    test_list.append(TickerData(timestamp=datetime.fromtimestamp(timestamp), exchange=my_dict.get("exchange"), symbol=my_dict.get("symbol"), ltp=my_dict.get("ltp"), close=my_dict.get("close"), open=my_dict.get("open"), high=my_dict.get("high"), low=my_dict.get("low"), vtt=my_dict.get("vtt"), total_buy_qty=my_dict.get("total_buy_qty"), lower_circuit=my_dict.get("lower_circuit"), upper_circuit=my_dict.get("upper_circuit"), bids=my_dict.get("bids"), ltt=datetime.fromtimestamp(ltt), asks=my_dict.get("asks")))
TickerData.objects.bulk_create(test_list)
end_time = datetime.now()
print(end_time-start_time)
len(test_list)

0:00:02.260029


6000

In [30]:
TickerData.objects.count()

23200